In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np

In [2]:
job_title = []
job_link = []

x = 1
i = 0

while x > 0:

    try:
        url = 'https://jobs.cisco.com/jobs/SearchJobs/?21178=%5B169482%5D&21178_format=6020&listFilterMode=1&projectOffset='+str(i*25)
    
        driver = webdriver.Chrome()
        driver.implicitly_wait(20)
        driver.get(url)

        table = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/table')

        for j in range(1,25):
            job_title.append(table.find_element_by_xpath('//*[@id="content"]/div/div[2]/table/tbody/tr['+str(j)+']/td[1]/a').text)
            job_link.append(table.find_element_by_xpath('//*[@id="content"]/div/div[2]/table/tbody/tr['+str(j)+']/td[1]/a').get_attribute('href'))
        driver.close()
        i += 1
    except:
        print('last page scraped: ', i)
        x -= 1
# Go to the below url to check how many total pages there are and compare with the last page scraped value.
# https://jobs.cisco.com/jobs/SearchJobs/?21178=%5B169482%5D&21178_format=6020&listFilterMode=1&projectOffset=0

last page scraped:  18


In [3]:
print(len(job_title), len(job_link))

446 446


In [4]:
df = pd.DataFrame(zip(job_title, job_link))
df.columns = ['job_title', 'job_link']
df.head()

,job_title,job_link
0,Senior Technical Program Manager,https://jobs.cisco.com/jobs/ProjectDetail/Seni...
1,"Technical Program Manager, Supply Chain",https://jobs.cisco.com/jobs/ProjectDetail/Tech...
2,Technical Product Owner,https://jobs.cisco.com/jobs/ProjectDetail/Tech...
3,Software Acceleration Sales Business Developme...,https://jobs.cisco.com/jobs/ProjectDetail/Soft...
4,Technical Consulting Engineer (1394769),https://jobs.cisco.com/jobs/ProjectDetail/Tech...


In [5]:
df['job_desc'] = np.nan
df.head()

,job_title,job_link,job_desc
0,Senior Technical Program Manager,https://jobs.cisco.com/jobs/ProjectDetail/Seni...,NaN
1,"Technical Program Manager, Supply Chain",https://jobs.cisco.com/jobs/ProjectDetail/Tech...,NaN
2,Technical Product Owner,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,NaN
3,Software Acceleration Sales Business Developme...,https://jobs.cisco.com/jobs/ProjectDetail/Soft...,NaN
4,Technical Consulting Engineer (1394769),https://jobs.cisco.com/jobs/ProjectDetail/Tech...,NaN


May work to remove these paragraphs
to avoid = ['Who we are', 'Cisco’s ONEx organization is', 'Message to applicants', 
            'When available, the salary range posted', 'At Cisco Meraki' ]

In [44]:
for i in range(len(df['job_link'])):
    try:

        url = (df['job_link'][i])
        driver = webdriver.Chrome()
        driver.get(url)

        txt = driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div/div[2]/div[3]/div/div[1]')
        desc = txt.find_elements_by_tag_name('li')
        df['job_desc'][i] = ''

        for d in desc:
            df['job_desc'][i] = df['job_desc'][i]+' '+d.text

        driver.close()
        
    except:
        df['job_desc'][i] = np.nan

In [49]:
df['job_desc'][2]

' You will work with Product Owners in the support of business processes & capabilities. Conduct deep analysis of various attributes within the Quote to Cash processes. Define, analyze and translate business requirements and Use Cases to provide functional requirements to IT Team, partnering with Product Managers, Business team members and SMEs. Identify operational improvement opportunities through analysis of business drivers and practices. Evaluate solutions in light of operational impacts and influence solutions that improve the quality of operational delivery and customer experience Align with business architecture business rules, policies, key business decisions, cross-program dependencies, and high-level solution designs. Support work intake process and end to end impact analysis for CRs. Drives capability co-design sessions, requirements assessment & evaluate IT solution on quality, and negotiate options Aligns with business architecture business rules, policies, critical busin

In [11]:
df.isna().sum()

job_title    0
job_link     0
job_desc     3
dtype: int64

In [12]:
df = df.dropna()
len(df)

443

In [13]:
df.sample(5)

,job_title,job_link,job_desc
137,Engineering Project Specialist - ThousandEyes,https://jobs.cisco.com/jobs/ProjectDetail/Engi...,Work on special projects with Engineering lea...
251,Federal Civilian Account Manager,https://jobs.cisco.com/jobs/ProjectDetail/Fede...,
397,"Designated Services Manager - NGFW, ISE (1385916)",https://jobs.cisco.com/jobs/ProjectDetail/Desi...,Identity Services Engines & Access Management...
293,Software Engineer- Fullstack Developer 1390720,https://jobs.cisco.com/jobs/ProjectDetail/Soft...,Create an outstanding secure device experienc...
88,Technical Consulting Engineer- 1391968,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,You have 4-6 years of relevant experience. Yo...


In [21]:
to_drop = df[df['job_desc'] == ''].index

In [22]:
df = df.drop(to_drop)
len(df)

379

In [58]:
df = df.reset_index(drop=True)
df.head()

,job_title,job_link,job_desc
0,Senior Technical Program Manager,https://jobs.cisco.com/jobs/ProjectDetail/Seni...,Drive end-to-end engineering delivery and own...
1,"Technical Program Manager, Supply Chain",https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Work with a cross-functional team across Supp...
2,Technical Product Owner,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,You will work with Product Owners in the supp...
3,Technical Consulting Engineer (1394769),https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Provides second/third level technical support...
4,Technical Consulting Engineer,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Bachelor's degree in a technical field or 5 y...


In [59]:
df.to_csv('cisco_jobs_usa.csv')

In [60]:
df.sample(5)

,job_title,job_link,job_desc
281,Security Research Engineer,https://jobs.cisco.com/jobs/ProjectDetail/Secu...,"Research security threats, attacker technique..."
254,"System Architect - Production Network, Google/WEB",https://jobs.cisco.com/jobs/ProjectDetail/Syst...,"The latest on relevant solutions, products an..."
160,Associate Hardware Engineer - Acacia,https://jobs.cisco.com/jobs/ProjectDetail/Asso...,"Experienced in reading and interpreting BOMs,..."
134,Technical Leader Software - Multicast Routing ...,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Proficient in C programming. Experience with ...
117,Sr. Software Engineer - Endpoint Health-Duo Se...,https://jobs.cisco.com/jobs/ProjectDetail/Sr-S...,"7+ years of development experience, especiall..."


In [10]:
df = pd.read_csv('cisco_jobs_usa.csv')
df.head()

,Unnamed: 0,job_title,job_link,job_desc
0,0,Senior Technical Program Manager,https://jobs.cisco.com/jobs/ProjectDetail/Seni...,Drive end-to-end engineering delivery and own...
1,1,"Technical Program Manager, Supply Chain",https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Work with a cross-functional team across Supp...
2,2,Technical Product Owner,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,You will work with Product Owners in the supp...
3,3,Technical Consulting Engineer (1394769),https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Provides second/third level technical support...
4,4,Technical Consulting Engineer,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Bachelor's degree in a technical field or 5 y...


In [11]:
df.columns = ['COMPANY', 'TITLE', 'LINK', 'QUALIFICATIONS']

In [13]:
df['COMPANY'] = 'CISCO'
df['DESCRIPTION'] = df['QUALIFICATIONS']
df.head()

,COMPANY,TITLE,LINK,QUALIFICATIONS,DESCRIPTION
0,CISCO,Senior Technical Program Manager,https://jobs.cisco.com/jobs/ProjectDetail/Seni...,Drive end-to-end engineering delivery and own...,Drive end-to-end engineering delivery and own...
1,CISCO,"Technical Program Manager, Supply Chain",https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Work with a cross-functional team across Supp...,Work with a cross-functional team across Supp...
2,CISCO,Technical Product Owner,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,You will work with Product Owners in the supp...,You will work with Product Owners in the supp...
3,CISCO,Technical Consulting Engineer (1394769),https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Provides second/third level technical support...,Provides second/third level technical support...
4,CISCO,Technical Consulting Engineer,https://jobs.cisco.com/jobs/ProjectDetail/Tech...,Bachelor's degree in a technical field or 5 y...,Bachelor's degree in a technical field or 5 y...


In [15]:
df.to_csv('cisco_jobs_usa.csv')